# VacationPy DAS
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os
import googlemaps
import time

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_csv_file = "../3_output_data/cities.csv"

city_csv_file = pd.read_csv("../3_output_data/cities.csv")

city_csv_file.head()

,city_name,country,date,latitude,longitude,temp,humidity,cloudiness,wind_speed
0,Karakol,KG,1612788365,42.4907,78.3936,35.42,78,100,3.31
1,Darhan,MN,1612788365,49.4867,105.9228,6.44,89,21,2.42
2,Sassandra,CI,1612788365,4.9500,-6.0833,83.14,77,23,6.78
3,Rantepao,ID,1612788366,-2.9701,119.8978,69.71,89,100,1.70
4,Jamestown,US,1612788366,42.0970,-79.2353,10.00,85,90,4.61


In [3]:
len(city_csv_file)

576

In [4]:
#type(city_csv_file)

In [5]:
#city_csv_file.columns

In [6]:
#Narrow down the cities to fit weather conditions
#Drop any rows will null values
#vacation_criteria_df = pd.DataFrame(city_data_df,  columns = ["City", "Country", "Max Temp", "Humidity", "Cloudiness"]
#new = old[['A', 'C', 'D']].copy().

vacation_criteria_df = city_csv_file[['city_name', 'country', 'temp', 'humidity', 'cloudiness', 'latitude', 'longitude' ]].copy()

vacation_city_df = vacation_criteria_df.loc[(vacation_criteria_df["cloudiness"]== 0) & (vacation_criteria_df["temp"] >= 70) & (vacation_criteria_df["temp"] <= 80)].dropna()

#vacation_city_df

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
len(vacation_city_df)

22

In [8]:
vacation_city_df = vacation_city_df[['city_name', 'country', 'temp', 'humidity', 'cloudiness', 'latitude', 'longitude']]

vacation_city_sort_df = vacation_city_df.sort_values(by = ['country', 'city_name'])

#Sorted by Country
#vacation_city_sort_df 

In [9]:
len(vacation_city_sort_df)

22

In [10]:
#vacation_city_sort_df.columns

### LOAD GOOGLE MAP¶



In [11]:
locations = vacation_city_sort_df[["latitude", "longitude"]]

#locations

In [12]:
gmaps.configure(api_key=g_key)

In [13]:
# locations
humidity = vacation_city_sort_df["humidity"].astype(float)

# Plot Heatmap
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Use the gmaps.figure passing a zoom_level of 2 and a center point so the map displays properly at
# a readable size
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))
fig = gmaps.figure()
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1.5)

# Add heat layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:

# locations
humidity = vacation_city_sort_df["humidity"].astype(float)
# humidity
figure_layout = {
    'width' : '1000px',
    'height':'400px',
    'border':'1px solid black',
    'padding':'1px',
    'margin':'0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=5,
                                   point_radius=1)
fig.add_layer(heatmap_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
hotel_df = vacation_city_sort_df

hotel_df["Hotel Name"] = ""

hotel_df = hotel_df.reset_index(drop=True)

#hotel_df

In [16]:
#creates a list to capture hotel_names to insert into dataframe
Hotel_Name = []

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

In [17]:
# taken from Airpost_Ratings Example done in class
for index, row in hotel_df.iterrows():
    # get lat, lng from df  
    lat = row["latitude"]
    lng = row["longitude"]
    city_name = row["city_name"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    
#Save the hotel name to the dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Could not find result")
        print("-------------------------")
        #delay the code by one second to avoid an error
        time.sleep(1)
    print("--------------------Ending Search")

Retrieving Results for Index 0: Dubai.
Closest hotel in Dubai is Rove La Mer Beach.
--------------------Ending Search
Retrieving Results for Index 1: Esperance.
Closest hotel in Esperance is The Jetty Resort.
--------------------Ending Search
Retrieving Results for Index 2: Cidreira.
Closest hotel in Cidreira is Hotel Castelo.
--------------------Ending Search
Retrieving Results for Index 3: Itapeva.
Closest hotel in Itapeva is Ibiza Hotel.
--------------------Ending Search
Retrieving Results for Index 4: Jatai.
Closest hotel in Jatai is Thile Park Hotel.
--------------------Ending Search
Retrieving Results for Index 5: Vallenar.
Closest hotel in Vallenar is Hotel Del Marques.
--------------------Ending Search
Retrieving Results for Index 6: Ambur.
Closest hotel in Ambur is OYO 36586 Sai Dr Venkatesh Residency.
--------------------Ending Search
Retrieving Results for Index 7: Diu.
Closest hotel in Diu is Hotel TGH - The Grand Highness.
--------------------Ending Search
Retrieving Resul

In [19]:
hotel_df

,city_name,country,temp,humidity,cloudiness,latitude,longitude,Hotel Name
0,Dubai,AE,77.00,53,0,25.2582,55.3047,Rove La Mer Beach
1,Esperance,AU,78.80,44,0,-33.8667,121.9000,The Jetty Resort
2,Cidreira,BR,75.25,62,0,-30.1811,-50.2056,Hotel Castelo
3,Itapeva,BR,72.43,60,0,-23.9822,-48.8756,Patriarca Hotel
4,Jatai,BR,77.07,63,0,-17.8814,-51.7144,Hotel A&P
5,Vallenar,CL,70.12,45,0,-28.5708,-70.7581,Hotel Solaris
6,Ambur,IN,74.75,51,0,12.7833,78.7000,OYO 36586 Sai Dr Venkatesh Residency
7,Diu,IN,75.56,47,0,20.7141,70.9822,Hotel TGH - The Grand Highness
8,Nagapattinam,IN,76.35,69,0,10.7667,79.8333,
9,Pandharpur,IN,79.39,15,0,17.6667,75.3333,Hotel Vitthal Inn Pandharpur


In [20]:
len(hotel_df)

22

In [27]:
hotel_df = hotel_df[:10]
hotel_df

,city_name,country,temp,humidity,cloudiness,latitude,longitude,Hotel Name
0,Dubai,AE,77.00,53,0,25.2582,55.3047,Rove La Mer Beach
1,Esperance,AU,78.80,44,0,-33.8667,121.9000,The Jetty Resort
2,Cidreira,BR,75.25,62,0,-30.1811,-50.2056,Hotel Castelo
3,Itapeva,BR,72.43,60,0,-23.9822,-48.8756,Ibiza Hotel
4,Jatai,BR,77.07,63,0,-17.8814,-51.7144,Thile Park Hotel
5,Vallenar,CL,70.12,45,0,-28.5708,-70.7581,Hotel Del Marques
6,Ambur,IN,74.75,51,0,12.7833,78.7000,OYO 36586 Sai Dr Venkatesh Residency
7,Diu,IN,75.56,47,0,20.7141,70.9822,Hotel TGH - The Grand Highness
8,Nagapattinam,IN,76.35,69,0,10.7667,79.8333,
9,Pandharpur,IN,79.39,15,0,17.6667,75.3333,Hotel Vitthal Inn Pandharpur


In [28]:
#name_address

In [29]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city_name}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]


In [30]:
#hotel_info

In [31]:
target_coordinates = f"{hotel_df['latitude'][0]},{hotel_df['longitude'][0]}"
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [32]:
# Add marker layer ontop of heat map

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…